In [48]:
import torch
import tiktoken

batch_size = 3
tokenizer = tiktoken.encoding_for_model('gpt2')
prefixes = ['a', 'b', 'cc', 'ddddd']

prefixes = tokenizer.encode_batch(prefixes)
max_len = max([len(i) for i in prefixes])
padded_prefixes = [[tokenizer.eot_token] * (max_len - len(i)) + i for i in prefixes]

batch_rem = len(prefixes) % batch_size
batch_padding = torch.ones(size=(batch_size - batch_rem, max_len))
batch_padding = batch_padding.masked_fill(batch_padding == 1, tokenizer.eot_token)

padded_prefixes = torch.tensor(padded_prefixes)
batch_prefixes = torch.cat((padded_prefixes, batch_padding), dim = 0)
batch_prefixes = batch_prefixes.reshape(-1, batch_size, max_len)

attn_mask = torch.ones_like(batch_prefixes)
attn_mask = attn_mask.masked_fill(batch_prefixes == tokenizer.eot_token, 0)


In [49]:
batch_prefixes

tensor([[[50256., 50256.,    64.],
         [50256., 50256.,    65.],
         [50256., 50256.,   535.]],

        [[ 1860.,  1860.,    67.],
         [50256., 50256., 50256.],
         [50256., 50256., 50256.]]])

In [50]:
attn_mask

tensor([[[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [31]:
padded_prefixes = torch.tensor(padded_prefixes)

In [33]:
padded_prefixes.shape

torch.Size([4, 3])

tensor([[[50256., 50256.,    64.],
         [50256., 50256.,    65.],
         [50256., 50256.,   535.]],

        [[ 1860.,  1860.,    67.],
         [50256., 50256., 50256.],
         [50256., 50256., 50256.]]])

In [44]:
attn_mask = torch.ones_like(batch_prefixes)
attn_mask = attn_mask.masked_fill(batch_prefixes == tokenizer.eot_token, 0)

In [45]:
attn_mask

tensor([[[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]]])